# Appmode in Jupyter

This page demonstrates how to generate online "apps" with a Jupyter Notebook interface. With the `appmode` plugin, we can create interactive experiences without requiring coding or running a specific cell.

Markdown cells will be retained, and all code cells will be run, then hidden. The outputs of each cell will be displayed as well.

Check out a further example using an [ipyvolume](https://ipyvolume.readthedocs.io/en/latest/) to render a 3D plot: [ipyvolume_demo.ipynb](ipyvolume_demo.ipynb)

In [6]:
import plotly.graph_objects as go
import ipywidgets as ipw
from ipywidgets import interactive_output, interact, Layout

#widget_style = {'description_width': 'initial'}
widget_style = {
    'layout': {'width': '400px'},
    'style': {'description_width': '60%'}
}

controls = {
    'num_sophisticated_investors': ipw.BoundedIntText(
        value=5,
        min=0,
        max=10,
        description='Number of sophisticated investors',
        **widget_style,
    ),
    'sophisticated_investor_average_value': ipw.IntSlider(
        value=100000,
        min=0,
        max=500000,
        description='Average donation per sophisticated investor',
        **widget_style,
    ),
    'num_direct_investors': ipw.BoundedIntText(
        value=5,
        min=0,
        max=10,
        description='Number of sophisticated investors',
        **widget_style,
    ),
    'direct_investor_average_value': ipw.IntSlider(
        value=100000,
        min=0,
        max=500000,
        description='Average donation per direct sophisticated investor',
        **widget_style,
    ),
    'num_community_investors': ipw.BoundedIntText(
        value=50,
        min=0,
        max=500,
        description='Number of community investors',
        **widget_style,
    ),
    'community_investor_average_value': ipw.IntSlider(
        value=1000,
        min=0,
        max=10000,
        description='Average donation per community investor',
        **widget_style,
    ),
    'community_capital_platform_fee_type': ipw.Dropdown(
        options=['Percentage', 'Flat rate'],
        value='Percentage',
        description='Type of crowdfunding platform fee:',
        **widget_style,
    ),
    'community_capital_platform_fee_percentage': ipw.widgets.FloatSlider(
        value=0.03,
        min=0,
        max=0.5,
        step=0.01,
        description='Percentage charged by crowdfunding platform',
        **widget_style,
    ),
    'property_purchase_cost': ipw.widgets.IntSlider(
        value=700000,
        min=100000,
        max=1500000,
        step=5000,
        description='Property purchase cost',
        **widget_style,
    ),
    'property_purchase_overhead': ipw.widgets.IntSlider(
        value=10000,
        min=1000,
        max=25000,
        step=500,
        description='Property purchase overhead',
        **widget_style,
    ),
    'buildout_cost': ipw.widgets.IntSlider(
        value=100000,
        min=1000,
        max=250000,
        step=1000,
        description='Property buildout cost',
        **widget_style,
    ),
}

ui = ipw.HBox([
    ipw.VBox([
#         ipw.Label('Funds raised'),
#         ipw.VBox([
            controls['num_sophisticated_investors'],
            controls['sophisticated_investor_average_value'],
#         ]),
#         ipw.VBox([
            controls['num_direct_investors'],
            controls['direct_investor_average_value'],
#         ]),
 #       ipw.VBox([
            controls['num_community_investors'],
            controls['community_investor_average_value'],
            controls['community_capital_platform_fee_type'],
            controls['community_capital_platform_fee_percentage'],
#        ]),
            
    ]),
    ipw.VBox([
#        ipw.Label('Costs'),
        controls['property_purchase_cost'],
        controls['property_purchase_overhead'],
        controls['buildout_cost'],
    ]),
])


def build_sankey_transactions(**kwargs):
    accounts = [ 
        'Total raise',
        'SEED Commons',
        'Costs',
    ]
    transactions = []
    
    """Sophisticated investors"""
    if 'num_sophisticated_investors' in kwargs and 'sophisticated_investor_average_value' in kwargs:
        for i in range(1,kwargs['num_sophisticated_investors']+1):
            accounts.append('Sophisticated investor {}'.format(i))
            
        for i in range(1,kwargs['num_sophisticated_investors']+1):
            transactions.append([accounts.index('Sophisticated investor {}'.format(i)),
                                 accounts.index('SEED Commons'),
                                 kwargs['sophisticated_investor_average_value']
                                ])
        transactions.append([
            accounts.index('SEED Commons'),
            accounts.index('Total raise'),
            kwargs['num_sophisticated_investors'] * kwargs['sophisticated_investor_average_value']
        ])
    
    """Direct investors"""
    if 'num_direct_investors' in kwargs and 'direct_investor_average_value' in kwargs:
        for i in range(1,kwargs['num_sophisticated_investors']+1):
            accounts.append('Direct investor {}'.format(i))
            
        for i in range(1,kwargs['num_direct_investors']+1):
            transactions.append([accounts.index('Direct investor {}'.format(i)),
                                 accounts.index('Total raise'),
                                 kwargs['direct_investor_average_value']
                                ])
            
    """Community capital"""
    if all((w in kwargs for w in ('num_community_investors', 
        'community_investor_average_value', 
        'community_capital_platform_fee_type', 
        'community_capital_platform_fee_percentage',
                                 ))):
        
        accounts.append('Community capital platform')
        accounts.append('Community capital platform fees')
        
        for i in range(1,kwargs['num_community_investors']+1):
            accounts.append('Community investor {}'.format(i))
            
        for i in range(1,kwargs['num_community_investors']+1):
            transactions.append([accounts.index('Community investor {}'.format(i)),
                                 accounts.index('Community capital platform'),
                                 kwargs['community_investor_average_value']
                                ])
            
        # TODO: switch between monthly/percentage fees
        cc_raised = kwargs['community_investor_average_value'] * kwargs['num_community_investors']
        cc_delivered = cc_raised * (1-kwargs['community_capital_platform_fee_percentage'])
        cc_fee = cc_raised - cc_delivered
        
        transactions.append([accounts.index('Community capital platform'),
                             accounts.index('Total raise'),
                             cc_delivered
                            ])
        transactions.append([accounts.index('Community capital platform'),
                             accounts.index('Community capital platform fees'),
                             cc_fee
                            ])
        transactions.append([accounts.index('Community capital platform fees'),
                             accounts.index('Costs'),
                             cc_fee
                            ])
        
    """Building costs"""
    accounts.append('Property purchase')
    accounts.append('Property purchase overhead')
    accounts.append('Buildout costs')
    
    transactions.append([accounts.index('Total raise'),
                         accounts.index('Property purchase'),
                         kwargs['property_purchase_cost']
                        ])
    transactions.append([accounts.index('Property purchase'),
                         accounts.index('Costs'),
                         kwargs['property_purchase_cost']
                        ])
    transactions.append([accounts.index('Total raise'),
                         accounts.index('Property purchase overhead'),
                         kwargs['property_purchase_overhead']
                        ])
    transactions.append([accounts.index('Property purchase overhead'),
                         accounts.index('Costs'),
                         kwargs['property_purchase_overhead']
                        ])
    transactions.append([accounts.index('Total raise'),
                         accounts.index('Buildout costs'),
                         kwargs['buildout_cost']
                        ])
    transactions.append([accounts.index('Buildout costs'),
                         accounts.index('Costs'),
                         kwargs['buildout_cost']
                        ])
        
        
    
    #from pprint import pprint; pprint( {'accounts': accounts, 'transactions': transactions} )
    show_sankey_transactions(accounts, transactions)

def show_sankey_transactions(accounts, transactions):
    links = {k:v for k,v in zip(('source','target','value'),zip(*transactions))}
    
    fig = go.Figure(data=[go.Sankey(
        node = dict(
          pad = 25,
          thickness = 20,
          line = dict(color = "black", width = 0.5),
          label = accounts,
          color = "blue"
        ),
        link = links,
        orientation='h',
    )])

    fig.update_layout(
        title_text="Startup Capital", 
        font_size=10,
        width=1000,
        height=1000,
    )
    fig.show()
        

#out = interactive_output(build_sankey_transactions, controls)
#display(ui,out)
interact(build_sankey_transactions, **controls)


interactive(children=(BoundedIntText(value=5, description='Number of sophisticated investors', layout=Layout(w…

<function __main__.build_sankey_transactions(**kwargs)>